# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Apr 24 2023 12:51PM,261670,10,HH-41542,Hush Hush,Released
1,Apr 24 2023 12:51PM,261670,10,HH-41547,Hush Hush,Released
2,Apr 24 2023 12:51PM,261670,10,HH-41548,Hush Hush,Released
3,Apr 24 2023 12:51PM,261670,10,HH-41549,Hush Hush,Released
4,Apr 24 2023 12:50PM,261669,10,Enova-11702,Emerginnova,Released
5,Apr 24 2023 12:50PM,261669,10,Enova-11703,Emerginnova,Released
6,Apr 24 2023 12:47PM,261667,19,MSP221061,"Methapharm, Inc.",Released
7,Apr 24 2023 12:44PM,261666,19,ADV80016264,"AdvaGen Pharma, Ltd",Released
8,Apr 24 2023 12:40PM,261664,22,646630,"NBTY Global, Inc.",Released
9,Apr 24 2023 12:38PM,261663,18,EMERSA-9413013,"Emersa Waterbox, LLC",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
37,261664,Released,1
38,261666,Released,1
39,261667,Released,1
40,261669,Released,2
41,261670,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
261664,NaN,NaN,1.0
261666,NaN,NaN,1.0
261667,NaN,NaN,1.0
261669,NaN,NaN,2.0
261670,NaN,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261602,0.0,0.0,1.0
261612,0.0,8.0,21.0
261618,10.0,8.0,10.0
261619,0.0,1.0,0.0
261621,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
261602,0,0,1
261612,0,8,21
261618,10,8,10
261619,0,1,0
261621,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261602,0,0,1
1,261612,0,8,21
2,261618,10,8,10
3,261619,0,1,0
4,261621,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,261602,,,1
1,261612,,8,21
2,261618,10,8,10
3,261619,,1,
4,261621,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Apr 24 2023 12:51PM,261670,10,Hush Hush
4,Apr 24 2023 12:50PM,261669,10,Emerginnova
6,Apr 24 2023 12:47PM,261667,19,"Methapharm, Inc."
7,Apr 24 2023 12:44PM,261666,19,"AdvaGen Pharma, Ltd"
8,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc."
9,Apr 24 2023 12:38PM,261663,18,"Emersa Waterbox, LLC"
10,Apr 24 2023 12:36PM,261662,18,"Emersa Waterbox, LLC"
11,Apr 24 2023 12:30PM,261661,10,"Uniderm USA, Inc"
12,Apr 24 2023 12:29PM,261659,18,"Emersa Waterbox, LLC"
13,Apr 24 2023 12:29PM,261660,10,"ClearSpec, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Apr 24 2023 12:51PM,261670,10,Hush Hush,,,4
1,Apr 24 2023 12:50PM,261669,10,Emerginnova,,,2
2,Apr 24 2023 12:47PM,261667,19,"Methapharm, Inc.",,,1
3,Apr 24 2023 12:44PM,261666,19,"AdvaGen Pharma, Ltd",,,1
4,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",,,1
5,Apr 24 2023 12:38PM,261663,18,"Emersa Waterbox, LLC",,,1
6,Apr 24 2023 12:36PM,261662,18,"Emersa Waterbox, LLC",,,1
7,Apr 24 2023 12:30PM,261661,10,"Uniderm USA, Inc",,,1
8,Apr 24 2023 12:29PM,261659,18,"Emersa Waterbox, LLC",,,1
9,Apr 24 2023 12:29PM,261660,10,"ClearSpec, LLC",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 12:51PM,261670,10,Hush Hush,4,,
1,Apr 24 2023 12:50PM,261669,10,Emerginnova,2,,
2,Apr 24 2023 12:47PM,261667,19,"Methapharm, Inc.",1,,
3,Apr 24 2023 12:44PM,261666,19,"AdvaGen Pharma, Ltd",1,,
4,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",1,,
5,Apr 24 2023 12:38PM,261663,18,"Emersa Waterbox, LLC",1,,
6,Apr 24 2023 12:36PM,261662,18,"Emersa Waterbox, LLC",1,,
7,Apr 24 2023 12:30PM,261661,10,"Uniderm USA, Inc",1,,
8,Apr 24 2023 12:29PM,261659,18,"Emersa Waterbox, LLC",1,,
9,Apr 24 2023 12:29PM,261660,10,"ClearSpec, LLC",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 12:51PM,261670,10,Hush Hush,4,,
1,Apr 24 2023 12:50PM,261669,10,Emerginnova,2,,
2,Apr 24 2023 12:47PM,261667,19,"Methapharm, Inc.",1,,
3,Apr 24 2023 12:44PM,261666,19,"AdvaGen Pharma, Ltd",1,,
4,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 12:51PM,261670,10,Hush Hush,4.0,NaN,NaN
1,Apr 24 2023 12:50PM,261669,10,Emerginnova,2.0,NaN,NaN
2,Apr 24 2023 12:47PM,261667,19,"Methapharm, Inc.",1.0,NaN,NaN
3,Apr 24 2023 12:44PM,261666,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
4,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Apr 24 2023 12:51PM,261670,10,Hush Hush,4.0,0.0,0.0
1,Apr 24 2023 12:50PM,261669,10,Emerginnova,2.0,0.0,0.0
2,Apr 24 2023 12:47PM,261667,19,"Methapharm, Inc.",1.0,0.0,0.0
3,Apr 24 2023 12:44PM,261666,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
4,Apr 24 2023 12:40PM,261664,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3139757,46.0,9.0,0.0
15,784904,39.0,20.0,10.0
16,261602,1.0,0.0,0.0
18,1569914,5.0,1.0,0.0
19,784984,3.0,0.0,0.0
20,2354793,58.0,1.0,0.0
21,261657,0.0,1.0,0.0
22,261664,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3139757,46.0,9.0,0.0
1,15,784904,39.0,20.0,10.0
2,16,261602,1.0,0.0,0.0
3,18,1569914,5.0,1.0,0.0
4,19,784984,3.0,0.0,0.0
5,20,2354793,58.0,1.0,0.0
6,21,261657,0.0,1.0,0.0
7,22,261664,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,46.0,9.0,0.0
1,15,39.0,20.0,10.0
2,16,1.0,0.0,0.0
3,18,5.0,1.0,0.0
4,19,3.0,0.0,0.0
5,20,58.0,1.0,0.0
6,21,0.0,1.0,0.0
7,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,46.0
1,15,Released,39.0
2,16,Released,1.0
3,18,Released,5.0
4,19,Released,3.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,18,19,20,21,22
Status,,,,,,,,
Completed,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,9.0,20.0,0.0,1.0,0.0,1.0,1.0,0.0
Released,46.0,39.0,1.0,5.0,3.0,58.0,0.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,20.0,0.0,1.0,0.0,1.0,1.0,0.0
2,Released,46.0,39.0,1.0,5.0,3.0,58.0,0.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,18,19,20,21,22
0,Completed,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,9.0,20.0,0.0,1.0,0.0,1.0,1.0,0.0
2,Released,46.0,39.0,1.0,5.0,3.0,58.0,0.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()